In [3]:
%load_ext autoreload
%autoreload 2

from crossref.restful import Works, Journals
from pymedx_custom import CrossrefArticle
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
works = Works()

In [5]:
works_list =list(works.query(author='Kabashin'))

In [6]:
all_articles = [CrossrefArticle(work) for work in works_list]

In [7]:
df = pd.DataFrame([article.to_dict() for article in all_articles])

In [8]:
abstracts = df[df["type"] == "journal-article"]['abstract']

In [9]:
df[df["type"] == "journal-article"].notna().sum().sort_values(ascending=False)

publisher                 187
is_referenced_by_count    187
authors                   187
doi                       187
title                     187
type                      187
publication_date          187
issn                      187
journal                   187
volume                    184
link                      183
issue                     169
references                167
start_page                163
license                   143
end_page                   94
abstract                   75
dtype: int64

In [10]:
abstr_exmple = df[df['abstract'].notna()]['abstract']
abstr_exmple

11     An overview of research results related to the...
12     The concept of a photonic crystal waveguide-ba...
15     Pulsed radiation of CO2 laser has been used to...
17     Femtosecond laser radiation has been used to a...
23     A new mechanism of laser plasma generation of ...
                             ...                        
241    Dyes used in industries such as textile, paper...
243    Boron neutron capture therapy (BNCT) is one of...
244    Correction for ‘Tunable optical properties of ...
245    Control of the chemical composition and optica...
246    Proton therapy can treat tumors located in rad...
Name: abstract, Length: 78, dtype: object

In [17]:
df['authors'].explode().apply(lambda x: x.get('lastname')).unique().shape

(496,)